IMPLEMENTATION OF SIMUATLED ANNEALING ALGORITHM FOR 8 QUEENS PROBLEM

In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import time

In [5]:
html_code = """

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>3D Chessboard with 8 Queens - Simulated Annealing + CSP</title>
    <style>
      body {
        margin: 0;
        overflow: hidden;
        background: radial-gradient(ellipse at center, #0f0f1e 0%, #050510 100%);
        font-family: 'Georgia', serif;
      }
      canvas {
        display: block;
      }
      #info {
        position: absolute;
        top: 20px;
        left: 50%;
        transform: translateX(-50%);
        color: #ffd700;
        font-size: 22px;
        text-shadow: 0 0 10px rgba(255, 215, 0, 0.5);
        z-index: 100;
        pointer-events: none;
        letter-spacing: 2px;
      }
      #controls {
        position: absolute;
        bottom: 20px;
        right: 20px;
        color: #ffd700;
        z-index: 100;
        background: rgba(10, 10, 21, 0.88);
        padding: 12px;
        border-radius: 8px;
        border: 2px solid #ffd700;
        min-width: 220px;
      }
      #controls button, #controls select {
        background: #ffd700;
        color: #0a0a15;
        border: none;
        padding: 6px 10px;
        margin: 6px 0;
        cursor: pointer;
        font-family: 'Georgia', serif;
        font-size: 13px;
        border-radius: 4px;
        width: 100%;
        font-weight: bold;
      }
      #controls button:hover, #controls select:hover {
        background: #ffed4e;
      }
      #controls button:disabled {
        background: #666;
        cursor: not-allowed;
      }
      .stat {
        margin: 6px 0;
        font-size: 12px;
      }
      .label {
        color: #c0c0c0;
      }
      .value {
        color: #ffd700;
        font-weight: bold;
      }
      #small-controls {
        display:flex;
        gap:8px;
      }
      #small-controls button, #small-controls select { width: auto; flex:1; }
      #cspModeButtons {
        display: none;
        gap: 8px;
        margin-bottom: 8px;
      }
      #cspModeButtons button {
        flex: 1;
      }
      .mode-btn-active {
        background: #ffed4e !important;
        box-shadow: 0 0 10px rgba(255,215,0,0.5);
      }
      #placementHint {
        display: none;
        font-size: 11px;
        color: #ffed4e;
        margin: 8px 0;
        text-align: center;
      }
    </style>
  </head>
  <body>
    <div id="info">♛ 8 QUEENS - SIMULATED ANNEALING & CSP (Backtracking) ♛</div>

    <div id="controls">
      <select id="algoSelect" title="Choose algorithm">
        <option value="sa">Solve with Simulated Annealing</option>
        <option value="csp">Solve with CSP (Backtracking)</option>
      </select>

      <div id="cspModeButtons">
        <button id="autoSolveBtn">Auto Solve</button>
        <button id="manualPlaceBtn">Manual Placement</button>
      </div>

      <div id="placementHint">
        Click on board to place/remove queens
      </div>

      <div id="small-controls">
        <button id="solveBtn">Solve</button>
        <button id="stepBtn">Step</button>
      </div>

      <button id="resetBtn">Reset</button>

      <div class="stat">
        <span class="label" id="conflictsLabel">Conflicts:</span>
        <span class="value" id="conflicts">-</span>
      </div>
      <div class="stat" id="saTempRow" style="display:block;">
        <span class="label">Temperature:</span>
        <span class="value" id="temperature">-</span>
      </div>
      <div class="stat">
        <span class="label">Step:</span>
        <span class="value" id="step">-</span>
      </div>
      <div class="stat">
        <span class="label">Status:</span>
        <span class="value" id="status">Ready</span>
      </div>
    </div>

    <script type="module">
      import * as THREE from "https://cdn.jsdelivr.net/npm/three@0.164.1/build/three.module.js";

      // ---------- three.js scene ----------
      const scene = new THREE.Scene();
      scene.background = new THREE.Color(0x0a0a15);
      scene.fog = new THREE.FogExp2(0x0a0a15, 0.02);
      const camera = new THREE.PerspectiveCamera(45, window.innerWidth / window.innerHeight, 0.1, 1000);
      const renderer = new THREE.WebGLRenderer({ antialias: true });
      renderer.setSize(window.innerWidth, window.innerHeight);
      renderer.setPixelRatio(window.devicePixelRatio);
      renderer.shadowMap.enabled = true;
      renderer.shadowMap.type = THREE.PCFSoftShadowMap;
      document.body.appendChild(renderer.domElement);

      const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
      scene.add(ambientLight);

      const mainLight = new THREE.DirectionalLight(0xffffff, 1.2);
      mainLight.position.set(5, 15, 5);
      mainLight.castShadow = true;
      mainLight.shadow.mapSize.width = 2048;
      mainLight.shadow.mapSize.height = 2048;
      scene.add(mainLight);

      const fillLight = new THREE.DirectionalLight(0xffffff, 0.4);
      fillLight.position.set(-5, 10, -5);
      scene.add(fillLight);

      const boardSize = 8;
      const tileSize = 1;
      const board = new THREE.Group();

      const whiteMat = new THREE.MeshStandardMaterial({
        color: 0xf5deb3,
        roughness: 0.2,
        metalness: 0.3,
        emissive: 0x332211,
        emissiveIntensity: 0.03
      });
      const blackMat = new THREE.MeshStandardMaterial({
        color: 0x3d2817,
        roughness: 0.3,
        metalness: 0.4,
        emissive: 0x110805,
        emissiveIntensity: 0.06
      });

      const borderGeometry = new THREE.BoxGeometry(boardSize + 0.6, 0.3, boardSize + 0.6);
      const borderMaterial = new THREE.MeshStandardMaterial({
        color: 0x2c1810,
        roughness: 0.6,
        metalness: 0.2
      });
      const border = new THREE.Mesh(borderGeometry, borderMaterial);
      border.position.y = -0.1;
      border.castShadow = true;
      border.receiveShadow = true;
      board.add(border);

      const inlayGeometry = new THREE.BoxGeometry(boardSize + 0.5, 0.32, boardSize + 0.5);
      const inlayMaterial = new THREE.MeshStandardMaterial({
        color: 0xffd700,
        roughness: 0.3,
        metalness: 0.8,
        emissive: 0xffd700,
        emissiveIntensity: 0.02
      });
      const inlay = new THREE.Mesh(inlayGeometry, inlayMaterial);
      inlay.position.y = -0.09;
      board.add(inlay);

      const tileMeshes = [];

      for (let i = 0; i < boardSize; i++) {
        for (let j = 0; j < boardSize; j++) {
          const geometry = new THREE.BoxGeometry(tileSize, 0.15, tileSize);
          const material = (i + j) % 2 === 0 ? whiteMat : blackMat;
          const tile = new THREE.Mesh(geometry, material);
          tile.position.set(i - boardSize / 2 + 0.5, 0.025, j - boardSize / 2 + 0.5);
          tile.receiveShadow = true;
          tile.castShadow = true;
          board.add(tile);
          tile.userData = { row: j, col: i };
          tileMeshes.push(tile);
        }
      }
      scene.add(board);

      const createQueen = () => {
        const queenGroup = new THREE.Group();

        const queenMaterial = new THREE.MeshStandardMaterial({
          color: 0xffd700,
          metalness: 0.95,
          roughness: 0.15,
          envMapIntensity: 1.0
        });

        const accentMaterial = new THREE.MeshStandardMaterial({
          color: 0xc0c0c0,
          metalness: 0.9,
          roughness: 0.2
        });

        const baseGroup = new THREE.Group();

        const platformGeometry = new THREE.CylinderGeometry(0.38, 0.42, 0.08, 64);
        const platform = new THREE.Mesh(platformGeometry, accentMaterial);
        platform.position.y = 0.04;
        platform.castShadow = true;
        baseGroup.add(platform);

        const baseGeometry = new THREE.CylinderGeometry(0.32, 0.38, 0.18, 64);
        const base = new THREE.Mesh(baseGeometry, queenMaterial);
        base.position.y = 0.17;
        base.castShadow = true;
        baseGroup.add(base);

        const baseRingGeometry = new THREE.TorusGeometry(0.33, 0.025, 16, 64);
        const baseRing = new THREE.Mesh(baseRingGeometry, accentMaterial);
        baseRing.position.y = 0.26;
        baseRing.rotation.x = Math.PI / 2;
        baseRing.castShadow = true;
        baseGroup.add(baseRing);

        const lowerNeckGeometry = new THREE.CylinderGeometry(0.18, 0.28, 0.25, 64);
        const lowerNeck = new THREE.Mesh(lowerNeckGeometry, queenMaterial);
        lowerNeck.position.y = 0.425;
        lowerNeck.castShadow = true;
        baseGroup.add(lowerNeck);

        const midGeometry = new THREE.CylinderGeometry(0.14, 0.18, 0.3, 64);
        const mid = new THREE.Mesh(midGeometry, queenMaterial);
        mid.position.y = 0.675;
        mid.castShadow = true;
        baseGroup.add(mid);

        [0.55, 0.8].forEach(yPos => {
          const bandGeometry = new THREE.TorusGeometry(0.15, 0.02, 16, 64);
          const band = new THREE.Mesh(bandGeometry, accentMaterial);
          band.position.y = yPos;
          band.rotation.x = Math.PI / 2;
          band.castShadow = true;
          baseGroup.add(band);
        });

        queenGroup.add(baseGroup);

        const headGroup = new THREE.Group();
        headGroup.position.y = 0.82;

        const upperNeckGeometry = new THREE.CylinderGeometry(0.12, 0.14, 0.2, 64);
        const upperNeck = new THREE.Mesh(upperNeckGeometry, queenMaterial);
        upperNeck.position.y = 0.1;
        upperNeck.castShadow = true;
        headGroup.add(upperNeck);

        const crownBaseGeometry = new THREE.CylinderGeometry(0.18, 0.2, 0.12, 32);
        const crownBase = new THREE.Mesh(crownBaseGeometry, queenMaterial);
        crownBase.position.y = 0.26;
        crownBase.castShadow = true;
        headGroup.add(crownBase);

        const crownBandGeometry = new THREE.TorusGeometry(0.19, 0.02, 16, 32);
        const crownBand = new THREE.Mesh(crownBandGeometry, accentMaterial);
        crownBand.position.y = 0.32;
        crownBand.rotation.x = Math.PI / 2;
        crownBand.castShadow = true;
        headGroup.add(crownBand);

        const pointCount = 5;
        for (let i = 0; i < pointCount; i++) {
          const angle = (i / pointCount) * Math.PI * 2;
          const radius = 0.17;

          const isCenterPoint = i === 0;
          const height = isCenterPoint ? 0.22 : 0.18;
          const baseWidth = 0.06;

          const pointGeometry = new THREE.ConeGeometry(baseWidth, height, 4);
          const crownPoint = new THREE.Mesh(pointGeometry, queenMaterial);

          const x = Math.cos(angle) * radius;
          const z = Math.sin(angle) * radius;
          crownPoint.position.set(x, 0.32 + height / 2, z);

          crownPoint.castShadow = true;
          headGroup.add(crownPoint);

          const tipGeometry = new THREE.SphereGeometry(0.03, 16, 16);
          const tip = new THREE.Mesh(tipGeometry, accentMaterial);
          tip.position.set(x, 0.32 + height, z);
          tip.castShadow = true;
          headGroup.add(tip);
        }

        queenGroup.add(headGroup);
        queenGroup.userData.headGroup = headGroup;

        return queenGroup;
      };

      const queens = [];
      for (let i = 0; i < 8; i++) {
        const queen = createQueen();
        queen.traverse((child) => {
          if (child.isMesh) child.castShadow = true;
        });
        board.add(queen);
        queens.push(queen);
      }

      const addBackgroundEffects = () => {
        const floorGeometry = new THREE.CircleGeometry(25, 64);
        const floorMaterial = new THREE.MeshStandardMaterial({
          color: 0x0a0a15,
          roughness: 0.1,
          metalness: 0.9,
          side: THREE.DoubleSide
        });
        const floor = new THREE.Mesh(floorGeometry, floorMaterial);
        floor.rotation.x = -Math.PI / 2;
        floor.position.y = -0.5;
        floor.receiveShadow = true;
        scene.add(floor);

        const particleCount = 200;
        const particles = new THREE.BufferGeometry();
        const positions = new Float32Array(particleCount * 3);
        const colors = new Float32Array(particleCount * 3);

        for (let i = 0; i < particleCount; i++) {
          positions[i * 3] = (Math.random() - 0.5) * 50;
          positions[i * 3 + 1] = Math.random() * 30;
          positions[i * 3 + 2] = (Math.random() - 0.5) * 50;

          const colorChoice = Math.random();
          if (colorChoice > 0.8) {
            colors[i * 3] = 0.25;
            colors[i * 3 + 1] = 0.41;
            colors[i * 3 + 2] = 0.88;
          } else if (colorChoice > 0.6) {
            colors[i * 3] = 0.54;
            colors[i * 3 + 1] = 0;
            colors[i * 3 + 2] = 0.54;
          } else {
            colors[i * 3] = 1;
            colors[i * 3 + 1] = 0.84;
            colors[i * 3 + 2] = 0;
          }
        }

        particles.setAttribute('position', new THREE.BufferAttribute(positions, 3));
        particles.setAttribute('color', new THREE.BufferAttribute(colors, 3));

        const particleMaterial = new THREE.PointsMaterial({
          size: 0.15,
          transparent: true,
          opacity: 0.6,
          vertexColors: true,
          blending: THREE.AdditiveBlending,
          sizeAttenuation: true
        });

        window.particleSystem = new THREE.Points(particles, particleMaterial);
        scene.add(window.particleSystem);
      };

      addBackgroundEffects();

      camera.position.set(8, 10, 8);
      camera.lookAt(0, 0, 0);

      // ---------- shared state + UI ----------
      let mouseX = 0, mouseY = 0;
      let targetRotationX = 0, targetRotationY = 0;
      const windowHalfX = window.innerWidth / 2;
      const windowHalfY = window.innerHeight / 2;

      document.addEventListener('mousemove', (event) => {
        mouseX = (event.clientX - windowHalfX) / 100;
        mouseY = (event.clientY - windowHalfY) / 100;
      });

      const algoSelect = document.getElementById('algoSelect');
      const solveBtn = document.getElementById('solveBtn');
      const stepBtn = document.getElementById('stepBtn');
      const resetBtn = document.getElementById('resetBtn');
      const conflictsEl = document.getElementById('conflicts');
      const tempEl = document.getElementById('temperature');
      const stepEl = document.getElementById('step');
      const statusEl = document.getElementById('status');
      const saTempRow = document.getElementById('saTempRow');
      const cspModeButtons = document.getElementById('cspModeButtons');
      const autoSolveBtn = document.getElementById('autoSolveBtn');
      const manualPlaceBtn = document.getElementById('manualPlaceBtn');
      const placementHint = document.getElementById('placementHint');

      function updateQueenPositionsFromState(state) {
        for (let row = 0; row < 8; row++) {
          const col = state[row];
          if (col === undefined || col === null) {
            queens[row].visible = false;
          } else {
            queens[row].visible = true;
            queens[row].position.x = col - boardSize / 2 + 0.5;
            queens[row].position.z = row - boardSize / 2 + 0.5;
          }
        }
      }

      function conflicts(state) {
        let conflict = 0;
        const n = state.length;
        for (let i = 0; i < n; i++) {
          if (state[i] === undefined) continue;
          for (let j = i + 1; j < n; j++) {
            if (state[j] === undefined) continue;
            if (state[i] === state[j] || Math.abs(state[i] - state[j]) === Math.abs(i - j)) {
              conflict++;
            }
          }
        }
        return conflict;
      }

      function highlightTileFor(row, col, on=true) {
        for (const t of tileMeshes) {
          if (t.userData.row === row && t.userData.col === col) {
            t.material.emissive = t.material.emissive || new THREE.Color(0x000000);
            t.material.emissive.setHex(on ? 0x004400 : 0x000000);
            return;
          }
        }
      }

      // ---------- Simulated Annealing solver ----------
      let saState = [];
      let saTemp = 100;
      let saStep = 0;
      let saRunning = false;
      let saAnimationSpeed = 90;

      function randomState() {
        return Array.from({ length: 8 }, () => Math.floor(Math.random() * 8));
      }

      function randomNeighbor(state) {
        const newState = [...state];
        const row = Math.floor(Math.random() * 8);
        let newCol = Math.floor(Math.random() * 8);
        while (newCol === state[row]) {
          newCol = Math.floor(Math.random() * 8);
        }
        newState[row] = newCol;
        return newState;
      }

      function saUpdateUI() {
        conflictsEl.textContent = conflicts(saState);
        tempEl.textContent = saTemp.toFixed(4);
        stepEl.textContent = saStep;
      }

      function saReset() {
        saState = randomState();
        saTemp = 100;
        saStep = 0;
        saRunning = false;
        updateQueenPositionsFromState(saState);
        saUpdateUI();
        statusEl.textContent = 'Ready (Simulated Annealing)';
        solveBtn.disabled = false;
        stepBtn.disabled = false;
      }

      function saPerformStep() {
        if (conflicts(saState) === 0) {
          statusEl.textContent = '✅ Solution Found (SA)!';
          saRunning = false;
          solveBtn.disabled = false;
          stepBtn.disabled = false;
          return false;
        }

        const neighbor = randomNeighbor(saState);
        const currentConflicts = conflicts(saState);
        const neighborConflicts = conflicts(neighbor);
        const delta = neighborConflicts - currentConflicts;

        if (delta < 0) {
          saState = neighbor;
        } else {
          const prob = Math.exp(-delta / saTemp);
          if (Math.random() < prob) {
            saState = neighbor;
          }
        }

        saTemp *= 0.95;
        saStep++;

        updateQueenPositionsFromState(saState);
        saUpdateUI();

        if (saTemp < 0.0001) {
          statusEl.textContent = '❌ Stopped (SA temp too low)';
          saRunning = false;
          solveBtn.disabled = false;
          stepBtn.disabled = false;
          return false;
        }

        return true;
      }

      function saSolveWithAnimation() {
        if (!saRunning) return;
        const cont = saPerformStep();
        if (cont && saRunning) setTimeout(saSolveWithAnimation, saAnimationSpeed);
      }

      // ---------- CSP Backtracking solver with manual placement ----------
      let cspState = new Array(8).fill(undefined);
      let cspInitialAssignment = {};
      let cspRunning = false;
      let cspPaused = false;
      let cspStepCount = 0;
      let cspStopRequested = false;
      let cspStepMode = false;
      let cspManualMode = false;

      function cspIsConsistent(state, row, col) {
        for (let r = 0; r < 8; r++) {
          if (state[r] === undefined) continue;
          const c = state[r];
          if (c === col) return false;
          if (Math.abs(r - row) === Math.abs(c - col)) return false;
        }
        return true;
      }

      function sleep(ms) {
        return new Promise(res => setTimeout(res, ms));
      }

      async function cspBacktrack(row) {
        if (cspStopRequested) return false;

        if (row >= 8) {
          return true;
        }

        // Skip rows that have initial assignment
        if (cspInitialAssignment.hasOwnProperty(row)) {
          return await cspBacktrack(row + 1);
        }

        for (let col = 0; col < 8; col++) {
          while (cspPaused && !cspStepMode) {
            await sleep(30);
            if (cspStopRequested) return false;
          }

          highlightTileFor(row, col, true);

          if (cspIsConsistent(cspState, row, col)) {
            cspState[row] = col;
            updateQueenPositionsFromState(cspState);
            cspStepCount++;
            conflictsEl.textContent = cspState.filter(v => v !== undefined).length + " placed";
            stepEl.textContent = cspStepCount;
            statusEl.textContent = '⚙️ Solving (CSP)...';
            await sleep(180);

            const ok = await cspBacktrack(row + 1);
            if (ok) {
              highlightTileFor(row, col, false);
              return true;
            }

            highlightTileFor(row, col, true);
            await sleep(120);
            cspState[row] = undefined;
            updateQueenPositionsFromState(cspState);
            cspStepCount++;
            conflictsEl.textContent = cspState.filter(v => v !== undefined).length + " placed";
            stepEl.textContent = cspStepCount;
            await sleep(120);
          }

          highlightTileFor(row, col, false);

          if (cspStepMode) {
            return false;
          }

          if (cspStopRequested) return false;
        }

        return false;
      }

      async function cspRun() {
        cspStopRequested = false;
        cspPaused = false;
        cspStepMode = false;
        cspRunning = true;
        solveBtn.disabled = true;
        stepBtn.disabled = true;
        statusEl.textContent = '⚙️ Solving (CSP)...';

        cspState = new Array(8).fill(undefined);
        
        // Apply initial assignment
        for (const row in cspInitialAssignment) {
          cspState[row] = cspInitialAssignment[row];
        }
        
        updateQueenPositionsFromState(cspState);
        conflictsEl.textContent = cspState.filter(v => v !== undefined).length + " placed";
        stepEl.textContent = cspStepCount = 0;

        const ok = await cspBacktrack(0);

        cspRunning = false;
        solveBtn.disabled = false;
        stepBtn.disabled = false;

        if (ok) {
          statusEl.textContent = '✅ Solution Found (CSP)!';
        } else if (cspStopRequested) {
          statusEl.textContent = 'Stopped (CSP)';
        } else {
          statusEl.textContent = 'No solution (CSP)';
        }
      }

      async function cspStep() {
        if (cspRunning && !cspStepMode) {
          return;
        }
        if (!cspRunning) {
          cspRunning = true;
          cspStepMode = true;
          cspPaused = false;
          cspStopRequested = false;
          cspState = new Array(8).fill(undefined);
          
          for (const row in cspInitialAssignment) {
            cspState[row] = cspInitialAssignment[row];
          }
          
          cspStepCount = 0;
          updateQueenPositionsFromState(cspState);
          conflictsEl.textContent = cspState.filter(v => v !== undefined).length + " placed";
          stepEl.textContent = cspStepCount;
          statusEl.textContent = 'Step Mode (CSP)';
          solveBtn.disabled = true;
        }
        await cspBacktrack(0);

        cspPaused = true;
        cspRunning = false;
        cspStepMode = false;
        solveBtn.disabled = false;
      }

      // Raycasting for board clicks
      const raycaster = new THREE.Raycaster();
      const mouse = new THREE.Vector2();

      function onBoardClick(event) {
        if (!cspManualMode || cspRunning) return;

        mouse.x = (event.clientX / window.innerWidth) * 2 - 1;
        mouse.y = -(event.clientY / window.innerHeight) * 2 + 1;

        raycaster.setFromCamera(mouse, camera);
        const intersects = raycaster.intersectObjects(tileMeshes);

        if (intersects.length > 0) {
          const tile = intersects[0].object;
          const row = tile.userData.row;
          const col = tile.userData.col;

          // Check if this row already has a queen
          if (cspInitialAssignment.hasOwnProperty(row)) {
            // Remove queen from this row
            delete cspInitialAssignment[row];
            statusEl.textContent = 'Click board to place queens';
          } else {
            // Check constraints before placing
            const tempState = new Array(8).fill(undefined);
            for (const r in cspInitialAssignment) {
              tempState[r] = cspInitialAssignment[r];
            }
            
            if (cspIsConsistent(tempState, row, col)) {
              // Valid placement
              cspInitialAssignment[row] = col;
              statusEl.textContent = 'Click board to place queens';
            } else {
              // Invalid placement - show error
              statusEl.textContent = '❌ Invalid placement! (conflicts detected)';
              // Flash the tile red
              for (const t of tileMeshes) {
                if (t.userData.row === row && t.userData.col === col) {
                  const originalEmissive = t.material.emissive.getHex();
                  t.material.emissive.setHex(0x880000);
                  setTimeout(() => {
                    t.material.emissive.setHex(originalEmissive);
                  }, 500);
                  break;
                }
              }
              setTimeout(() => {
                statusEl.textContent = 'Click board to place queens';
              }, 1500);
              return;
            }
          }

          // Update display
          cspState = new Array(8).fill(undefined);
          for (const r in cspInitialAssignment) {
            cspState[r] = cspInitialAssignment[r];
          }
          updateQueenPositionsFromState(cspState);
          conflictsEl.textContent = Object.keys(cspInitialAssignment).length + " placed";
        }
      }

      renderer.domElement.addEventListener('click', onBoardClick);

      // ---------- control wiring ----------
      function setAlgoUI(algo) {
        if (algo === 'sa') {
          conflictsLabel.textContent = "Conflicts:";
          saTempRow.style.display = 'block';
          cspModeButtons.style.display = 'none';
          placementHint.style.display = 'none';
          statusEl.textContent = 'Ready (Simulated Annealing)';
        } else {
          conflictsLabel.textContent = "Placed Queens:";
          saTempRow.style.display = 'none';
          cspModeButtons.style.display = 'flex';
          statusEl.textContent = 'Ready (CSP Backtracking)';
        }
      }

      algoSelect.addEventListener('change', (e) => {
        setAlgoUI(e.target.value);
        if (e.target.value === 'csp') {
          // Default to auto solve mode
          cspManualMode = false;
          autoSolveBtn.classList.add('mode-btn-active');
          manualPlaceBtn.classList.remove('mode-btn-active');
          placementHint.style.display = 'none';
        }
      });

      autoSolveBtn.addEventListener('click', () => {
        cspManualMode = false;
        autoSolveBtn.classList.add('mode-btn-active');
        manualPlaceBtn.classList.remove('mode-btn-active');
        placementHint.style.display = 'none';
        
        // Clear initial assignment
        cspInitialAssignment = {};
        cspState = new Array(8).fill(undefined);
        updateQueenPositionsFromState(cspState);
        conflictsEl.textContent = "0 placed";
        statusEl.textContent = 'Auto Solve Mode';
      });

      manualPlaceBtn.addEventListener('click', () => {
        cspManualMode = true;
        manualPlaceBtn.classList.add('mode-btn-active');
        autoSolveBtn.classList.remove('mode-btn-active');
        placementHint.style.display = 'block';
        
        // Clear and prepare for manual placement
        cspInitialAssignment = {};
        cspState = new Array(8).fill(undefined);
        updateQueenPositionsFromState(cspState);
        conflictsEl.textContent = "0 placed";
        statusEl.textContent = 'Click board to place queens';
      });

      solveBtn.addEventListener('click', () => {
        const chosen = algoSelect.value;
        if (chosen === 'sa') {
          if (saStep === 0) saReset();
          saRunning = true;
          statusEl.textContent = '⚙️ Solving... (SA)';
          solveBtn.disabled = true;
          stepBtn.disabled = true;
          saSolveWithAnimation();
        } else {
          cspRun();
        }
      });

      stepBtn.addEventListener('click', async () => {
        const chosen = algoSelect.value;
        if (chosen === 'sa') {
          if (saStep === 0) saReset();
          saPerformStep();
        } else {
          await cspStep();
        }
      });

      resetBtn.addEventListener('click', () => {
        cspStopRequested = true;
        cspRunning = false;
        cspPaused = false;
        cspStepMode = false;
        saRunning = false;

        if (algoSelect.value === 'csp') {
          cspInitialAssignment = {};
          cspState = new Array(8).fill(undefined);
          updateQueenPositionsFromState(cspState);
          conflictsEl.textContent = "0 placed";
          stepEl.textContent = "0";
          
          if (cspManualMode) {
            statusEl.textContent = 'Click board to place queens';
          } else {
            statusEl.textContent = 'Ready (CSP Backtracking)';
          }
        } else {
          saReset();
        }

        solveBtn.disabled = false;
        stepBtn.disabled = false;
      });

      // Initialize
      saReset();
      setAlgoUI(algoSelect.value);
      autoSolveBtn.classList.add('mode-btn-active');

      // ---------- animation loop ----------
      function animate() {
        requestAnimationFrame(animate);

        const time = Date.now() * 0.001;

        targetRotationY = mouseX * 0.0003;
        targetRotationX = mouseY * 0.0003;

        board.rotation.y += targetRotationY;
        board.rotation.x += targetRotationX;

        board.rotation.y *= 0.95;
        board.rotation.x *= 0.95;

        board.rotation.y += 0.002;

        queens.forEach((queen, index) => {
          const headGroup = queen.userData.headGroup;
          if (headGroup) {
            headGroup.rotation.y = Math.sin(time * 0.5 + index) * 0.1;
          }
        });

        if (window.particleSystem) {
          const positions = window.particleSystem.geometry.attributes.position.array;
          for (let i = 0; i < positions.length; i += 3) {
            positions[i + 1] += Math.sin(time + i) * 0.01;
            if (positions[i + 1] > 30) {
              positions[i + 1] = 0;
            }
          }
          window.particleSystem.geometry.attributes.position.needsUpdate = true;
          window.particleSystem.rotation.y = time * 0.05;
        }

        renderer.render(scene, camera);
      }
      animate();

      window.addEventListener("resize", () => {
        camera.aspect = window.innerWidth / window.innerHeight;
        camera.updateProjectionMatrix();
        renderer.setSize(window.innerWidth, window.innerHeight);
      });

    </script>
  </body>
</html>

"""

with open("eight_queens_3d.html", "w", encoding="utf-8") as f:
    f.write(html_code)
print("✅ File created: eight_queens_3d.html")

✅ File created: eight_queens_3d.html


In [6]:
import webbrowser
webbrowser.open("eight_queens_3d.html")

True